In [ ]:
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import time
from tqdm import tqdm
from lightgbm import LGBMClassifier

### 6개월치 proba 평균

In [ ]:
df_weather=pd.read_csv('기상데이터 0510 index X.csv')
df_weather.head()

,관측일자,temperature,temp_max,temp_min,rainfall,humidity,조사지역,연,월,일
0,1988-01-01,-2.4,-0.8,-4.4,0.5,64.0,서울,1988,1,1
1,1988-01-01,-2.6,0.5,-6.2,1.9,73.0,관악산,1988,1,1
2,1988-01-01,-4.1,-0.7,-8.6,0.0,66.0,춘천,1988,1,1
3,1988-01-01,-3.1,-1.0,-8.6,0.4,79.0,철원,1988,1,1
4,1988-01-01,3.7,6.5,0.1,0.0,57.0,속초,1988,1,1


In [ ]:
df_grain=pd.read_csv('작물추천_min_max_추가_데이터타입변경(코코아,밀 추가).csv')
del df_grain['Unnamed: 0']
df_grain.head()

,temperature,rainfall,temp_min,temp_max,label
0,20.879744,202.935536,18,28,rice
1,21.770462,226.655537,18,28,rice
2,23.004459,263.964248,18,28,rice
3,26.491096,242.864034,18,28,rice
4,20.130175,262.717340,18,28,rice


In [ ]:
df_grain['label'].value_counts()

wheat          101
cocoa          101
muskmelon      100
coffee         100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
rice           100
maize          100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
kidneybeans    100
chickpea       100
watermelon     100
Name: label, dtype: int64

In [ ]:
# 6개월치 데이터를 추합 return 습도 x
def predict_6m_proba(region, start_month, pred_year):
    df = pd.DataFrame()
    for mon in range(start_month, start_month+6):
        pred_data = OLS_model_predict(region, mon, pred_year)
        # print(pred_data)
        tmp_data =[]
        for data in pred_data :
            tmp_data.append(data[-1])
        # print(tmp_data)
        df_pred  = predict_crops_6m(tmp_data)
        df = pd.concat([df, df_pred])
    return df.groupby(df.index).mean().sort_values(by='확률', ascending=False)

def predict_crops_6m(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]
    xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.01, n_estimators = 10)  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    grain_data= grain_data.append({'temperature' :  data[0], 'temp_max' : data[1], 'temp_min' : data[2], 'rainfall' : data[3]*30} , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-1], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    # print(grain_mmx[-1])
    df_pred = pd.DataFrame(xgb.predict_proba(list(grain_mmx[-1])), columns=labelencoder.classes_).T
    df_pred.columns = ['확률']
    df_pred = df_pred.sort_values(by='확률',ascending=False)
    return_pred = df_pred
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -1]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([value, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [ ]:
predict_6m_proba('서울', 3, 2022)

,확률
wheat,0.065607
kidneybeans,0.057892
grapes,0.057516
lentil,0.057515
maize,0.057280
cocoa,0.054272
rice,0.050048
chickpea,0.050037
coconut,0.050015
pomegranate,0.050009


In [ ]:
df.groupby(df.index).mean().sort_values(by='확률', ascending=False).head(3)

,확률
cocoa,0.252553
wheat,0.180007
maize,0.173578


In [ ]:
for region in tqdm(df_weather['조사지역'].unique()) :
    print(f'{region}지역 추천 Top 3 작물')
    display(predict_6m_proba(region, 3, 2022).head(3))
    print('\n')

  0%|          | 0/73 [00:00<?, ?it/s]

서울지역 추천 Top 3 작물


,확률
cocoa,0.252553
wheat,0.180007
maize,0.173578


  1%|▏         | 1/73 [00:04<05:43,  4.77s/it]



관악산지역 추천 Top 3 작물


...

,확률
cocoa,0.301050
wheat,0.254634
rice,0.071254


 99%|█████████▊| 72/73 [05:52<00:04,  4.56s/it]



고창지역 추천 Top 3 작물


,확률
wheat,0.282840
cocoa,0.183927
maize,0.170499


100%|██████████| 73/73 [05:57<00:00,  4.89s/it]

### 모든지역 데이터를 한개의 df로 리턴해주는 함수

In [ ]:
# 모든지역 데이터 return
def predict_6m_all_region(start_month, pred_year):
    df = pd.DataFrame()
    for mon in range(start_month, start_month+6):
        region_data = []
        for region in df_weather['조사지역'].unique():
            pred_data = OLS_model_predict(region, mon, pred_year)
            tmp_data =[]
            for data in pred_data :
                tmp_data.append(data[-1])
            tmp_data.append(region)
            region_data.append(tmp_data)

        # print(tmp_data)
        df_pred  = predict_crops_6m(region_data)
        df = pd.concat([df, df_pred])
        df_gr = df.groupby(df.index).mean()
        for col in df_gr.columns :
            df_gr = df_gr.sort_values(by=col, ascending=False)
        
    return df_gr

def predict_crops_6m(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]
    xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.01, n_estimators = 10)  
    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    for value in data:
        grain_data= grain_data.append({'temperature' :  value[0], 'temp_max' : value[1], 'temp_min' : value[2], 'rainfall' : value[3]*30} , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-len(data)], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )
    xgb.fit(X_train, y_train)

    # print(grain_mmx[-1])
    return_pred = pd.DataFrame()
    for i, value in enumerate(data):
        df_pred = pd.DataFrame(xgb.predict_proba(list(grain_mmx[-(len(data)-i)])), columns=labelencoder.classes_).T
        df_pred.columns = [value[4]]
        # df_pred = df_pred.sort_values(by='확률',ascending=False)
        return_pred = pd.concat([return_pred, df_pred], axis=1)
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -2]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([region, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [ ]:
%time predict_6m_all_region(4, 2100)

CPU times: user 1min, sys: 785 ms, total: 1min
Wall time: 1min


,서울,관악산,춘천,철원,속초,원주,대관령,태백,인제,홍천,...,포항,울릉도,동해,영월,흑산도,동두천,문산,진도,상주,고창
cocoa,0.059480,0.054247,0.059933,0.059924,0.064017,0.059718,0.050513,0.062386,0.059332,0.059093,...,0.058667,0.060303,0.059578,0.059094,0.055828,0.059963,0.055870,0.055468,0.059718,0.059311
coconut,0.058855,0.050008,0.051418,0.051385,0.051197,0.051202,0.050559,0.049587,0.051002,0.050778,...,0.066332,0.051769,0.051040,0.050779,0.059854,0.051438,0.051611,0.051260,0.051186,0.059063
wheat,0.059290,0.057948,0.059407,0.059737,0.059201,0.059191,0.075301,0.065584,0.067205,0.066967,...,0.058141,0.060118,0.059045,0.066968,0.051507,0.059426,0.059963,0.059168,0.059192,0.058774
maize,0.050929,0.064160,0.058687,0.058645,0.058083,0.058471,0.057832,0.063402,0.057915,0.058071,...,0.057407,0.051769,0.065183,0.058072,0.051558,0.058322,0.058871,0.058460,0.058471,0.057656
pomegranate,0.050912,0.049991,0.051400,0.051407,0.051170,0.055311,0.050590,0.049617,0.050998,0.054887,...,0.057961,0.051743,0.051013,0.054887,0.051534,0.051419,0.051609,0.051188,0.055311,0.054894
rice,0.050967,0.066305,0.051435,0.051417,0.051269,0.051218,0.058488,0.057571,0.051034,0.050795,...,0.050203,0.051720,0.051113,0.050795,0.051588,0.051455,0.051643,0.051247,0.051218,0.050799
banana,0.058830,0.049960,0.051388,0.051361,0.051173,0.051172,0.050501,0.049530,0.050988,0.050749,...,0.050191,0.051726,0.051016,0.050749,0.051515,0.051408,0.051597,0.051181,0.051172,0.050791
chickpea,0.050940,0.050043,0.051428,0.051411,0.051248,0.051187,0.050646,0.049622,0.051026,0.050788,...,0.050137,0.051824,0.051091,0.050788,0.051587,0.051448,0.051637,0.051267,0.051212,0.050770
cotton,0.050907,0.049972,0.051371,0.051354,0.051165,0.051155,0.050536,0.049539,0.050970,0.050732,...,0.050092,0.051749,0.051009,0.050720,0.051547,0.051391,0.051580,0.051193,0.051143,0.050738
papaya,0.050891,0.049948,0.051356,0.051338,0.051150,0.051140,0.050499,0.049527,0.050955,0.050716,...,0.050089,0.051723,0.050994,0.050717,0.051512,0.051375,0.051564,0.051168,0.051140,0.050723


### xgbooster best_params 추출
- 시계열 데이터의 특성을 고려하여 해당월의 classifier를 다 따로 돌림

In [ ]:
# OLS_model_predict(region, month, pred_year):
region = '서울'
month = 6
pred_year = 2022

df_tmp_region = df_weather[df_weather['조사지역'] == region]
df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
df_weather_region = df_weather_region.reset_index(drop=False)

df_group_mean = df_weather_region.groupby(['연', '월']).mean()
df_group_mean = df_group_mean.reset_index(drop=False)
df_month_mean = df_group_mean[df_group_mean['월'] == month]

for i in df_month_mean.index:
    df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
    temp_max = df_tmp['temp_max'].max()
    temp_min = df_tmp['temp_min'].min()
    df_month_mean.loc[i, 'temp_max'] = temp_max
    df_month_mean.loc[i, 'temp_min'] = temp_min

pred_list = []
for value in df_month_mean.columns[2: -1]:
    df_target = df_month_mean[value]
    X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
    y_train2 = df_target.values

    # import statsmodels.api as sm
    X_train = sm.add_constant(X_train2)
    model = sm.OLS(y_train2, X_train2).fit()
    pred_value = model.predict(pred_year)
    pred_list.append([value, float(pred_value)])

# print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
# print(pred_list)
pred_list

[['temperature', 22.80263004569429],
 ['temp_max', 32.66101643862828],
 ['temp_min', 14.510903067907272],
 ['rainfall', 4.682851271821746],
 ['humidity', 67.1027178427563]]

In [ ]:
from sklearn.model_selection import GridSearchCV
XGBClassifier().get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [ ]:
pred_data = pred_list
tmp_data =[]
for data in pred_data :
    tmp_data.append(data[-1])

data = tmp_data
tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]



labelencoder = LabelEncoder()
target_lb = labelencoder.fit_transform(tmp_grain.label)
scaler = MinMaxScaler()
grain_data = tmp_grain[tmp_grain.columns[:-1]]

grain_data= grain_data.append({'temperature' :  data[0], 'temp_max' : data[1], 'temp_min' : data[2], 'rainfall' : data[3]*30} , ignore_index=True)

grain_mmx = scaler.fit_transform(grain_data)

# display(grain_data.tail())
# display(grain_mmx[:-1])
# display(grain_mmx[-1])

X_train, X_test, y_train, y_test = train_test_split(
    grain_mmx[:-1], target_lb, stratify=target_lb, test_size=0.2,
    random_state=2022
)

xgb = XGBClassifier()  
params = {
    'booster' : ['gbtree', 'gblinear'],
    'n_estimators' : [10, 100, 1000],
    'learning_rate' : [0.01, 0.05, 0.1]
}

xgb_grid = GridSearchCV(
    xgb, params, scoring='accuracy', cv=5
)


xgb_grid.fit(X_train, y_train)

print(xgb_grid.best_score_, xgb_grid.best_params_)
xgb_best = xgb_grid.best_estimator_

# print(grain_mmx[-1])
df_pred = pd.DataFrame(xgb_best.predict_proba(list(grain_mmx[-1])), columns=labelencoder.classes_).T
df_pred.columns = ['확률']
df_pred = df_pred.sort_values(by='확률',ascending=False)
return_pred = df_pred
return_pred

0.9953968938740294 {'booster': 'gbtree', 'learning_rate': 0.01, 'n_estimators': 10}


,확률
cocoa,0.076987
coconut,0.051366
cotton,0.051278
banana,0.051278
maize,0.051278
papaya,0.051275
kidneybeans,0.051274
watermelon,0.051274
chickpea,0.051274
lentil,0.051273


In [ ]:
# xgbooster GridSearch 샘플 10개만 (너무 오래 걸림)
def xgboost_best_params(start_month, pred_year):
    sample_list = np.random.choice(72, 10)
    df = pd.DataFrame()
    for mon in range(start_month, start_month+6):
        region_data = []
        for region in df_weather['조사지역'].unique()[sample_list]:
            pred_data = OLS_model_predict(region, mon, pred_year)
            tmp_data =[]
            for data in pred_data :
                tmp_data.append(data[-1])
            tmp_data.append(region)
            region_data.append(tmp_data)

        # print(tmp_data)
        df_pred  = xgboost_params(region_data)
        df = pd.concat([df, df_pred])
        df_gr = df.groupby(df.index).mean()
        for col in df_gr.columns :
            df_gr = df_gr.sort_values(by=col, ascending=False)
        
    return df_gr

def xgboost_params(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]

    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    for value in data:
        grain_data= grain_data.append({'temperature' :  value[0], 'temp_max' : value[1], 'temp_min' : value[2], 'rainfall' : value[3]*30} , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-len(data)], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )


    xgb = XGBClassifier()  
    params = {
        'booster' : ['gbtree', 'gblinear'],
        'n_estimators' : [10, 100, 1000],
        'learning_rate' : [0.01, 0.05, 0.1]
    }

    xgb_grid = GridSearchCV(
        xgb, params, scoring='accuracy', cv=5
    )

    xgb_grid.fit(X_train, y_train)

    print(xgb_grid.best_score_, xgb_grid.best_params_)
    xgb_best = xgb_grid.best_estimator_

    # print(grain_mmx[-1])
    return_pred = pd.DataFrame()
    for i, value in enumerate(data):
        df_pred = pd.DataFrame(xgb_best.predict_proba(list(grain_mmx[-(len(data)-i)])), columns=labelencoder.classes_).T
        df_pred.columns = [value[4]]
        # df_pred = df_pred.sort_values(by='확률',ascending=False)
        return_pred = pd.concat([return_pred, df_pred], axis=1)
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -2]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([region, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [ ]:
xgboost_best_params(3, 2022)

0.9953968938740294 {'booster': 'gbtree', 'learning_rate': 0.01, 'n_estimators': 10}
0.9953968938740294 {'booster': 'gbtree', 'learning_rate': 0.01, 'n_estimators': 10}
0.9953968938740294 {'booster': 'gbtree', 'learning_rate': 0.01, 'n_estimators': 10}
0.9953968938740294 {'booster': 'gbtree', 'learning_rate': 0.01, 'n_estimators': 10}
0.9953968938740294 {'booster': 'gbtree', 'learning_rate': 0.01, 'n_estimators': 10}
0.9953968938740294 {'booster': 'gbtree', 'learning_rate': 0.01, 'n_estimators': 10}


,부안,속초,장수,안동,보령,완도,전주,장흥,울산,고산
coconut,0.050373,0.050752,0.049754,0.050409,0.050406,0.057320,0.050818,0.050766,0.050820,0.059031
kidneybeans,0.058251,0.050734,0.057633,0.050391,0.050386,0.049371,0.050785,0.058645,0.050787,0.058960
wheat,0.066327,0.067120,0.074164,0.066429,0.066392,0.065633,0.067222,0.067120,0.067225,0.058618
grapes,0.050322,0.058626,0.049704,0.050373,0.057906,0.057250,0.050767,0.050716,0.050769,0.058527
papaya,0.050326,0.050721,0.049708,0.050378,0.050359,0.049344,0.050772,0.050720,0.050774,0.055723
chickpea,0.050396,0.050817,0.049803,0.050448,0.050429,0.049415,0.050843,0.050791,0.050845,0.050719
maize,0.057638,0.058033,0.063938,0.064586,0.057670,0.056656,0.064981,0.058033,0.064984,0.050691
coffee,0.050357,0.050752,0.049741,0.050409,0.050423,0.051522,0.050804,0.050752,0.050806,0.050688
cocoa,0.054630,0.055024,0.058286,0.054682,0.054658,0.053642,0.055075,0.055022,0.055077,0.050676
pomegranate,0.050392,0.050788,0.049774,0.050444,0.050425,0.049387,0.050839,0.050787,0.050817,0.050665


### LGBMClassifier 사용

In [ ]:
# lgbm GridSearch 샘플 10개만 (너무 오래 걸림)
def lgbm_best_params(start_month, pred_year):
    sample_list = np.random.choice(72, 10)
    df = pd.DataFrame()
    for mon in range(start_month, start_month+6):
        region_data = []
        for region in df_weather['조사지역'].unique()[sample_list]:
            pred_data = OLS_model_predict(region, mon, pred_year)
            tmp_data =[]
            for data in pred_data :
                tmp_data.append(data[-1])
            tmp_data.append(region)
            region_data.append(tmp_data)

        # print(tmp_data)
        df_pred  = lgbm_params(region_data)
        df = pd.concat([df, df_pred])
        df_gr = df.groupby(df.index).mean()
        for col in df_gr.columns :
            df_gr = df_gr.sort_values(by=col, ascending=False)
        
    return df_gr

def lgbm_params(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]

    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    for value in data:
        grain_data= grain_data.append({'temperature' :  value[0], 'temp_max' : value[1], 'temp_min' : value[2], 'rainfall' : value[3]*30} , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-len(data)], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )


    lgb = LGBMClassifier() 
    params = {
        # 'boosting ' : ['gbdt', 'rf'],
        'n_estimators' : [100, 1000, 1000],
        'learning_rate' : [0.01, 0.05],
        'num_iteration ' : [5, 10, 50]
    }

    lgb_grid = GridSearchCV(
        lgb, params, scoring='accuracy', cv=5
    )



    evals = [(X_test, y_test)]
    lgb_grid.fit(X_train, y_train, eval_set=evals, eval_metric='logloss', verbose=False)

    print(lgb_grid.best_score_, lgb_grid.best_params_)
    lgb_best = lgb_grid.best_estimator_

    # print(grain_mmx[-1])
    return_pred = pd.DataFrame()
    for i, value in enumerate(data):
        df_pred = pd.DataFrame(lgb_best.predict_proba(list(grain_mmx[-(len(data)-i)])), columns=labelencoder.classes_).T
        df_pred.columns = [value[4]]
        # df_pred = df_pred.sort_values(by='확률',ascending=False)
        return_pred = pd.concat([return_pred, df_pred], axis=1)
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -2]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([region, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [ ]:
lgbm_best_params(4, 2022)

0.9980263157894737 {'learning_rate': 0.01, 'n_estimators': 1000, 'num_iteration ': 5}


ValueError: ignored

In [ ]:
# 모든지역 데이터 return
def predict_6m_all_region_lgb(start_month, pred_year):
    df = pd.DataFrame()
    for mon in range(start_month, start_month+6):
        region_data = []
        for region in df_weather['조사지역'].unique():
            pred_data = OLS_model_predict(region, mon, pred_year)
            tmp_data =[]
            for data in pred_data :
                tmp_data.append(data[-1])
            tmp_data.append(region)
            region_data.append(tmp_data)

        # print(tmp_data)
        df_pred  = predict_crops_6m(region_data)
        df = pd.concat([df, df_pred])
        df_gr = df.groupby(df.index).mean()
        for col in df_gr.columns :
            df_gr = df_gr.sort_values(by=col, ascending=False)
        
    return df_gr

def predict_crops_6m_lgb(data):
    tmp_grain = df_grain[['temperature', 'temp_max', 'temp_min',  'rainfall', 'label']]

    labelencoder = LabelEncoder()
    target_lb = labelencoder.fit_transform(tmp_grain.label)
    scaler = MinMaxScaler()
    grain_data = tmp_grain[tmp_grain.columns[:-1]]

    for value in data:
        grain_data= grain_data.append({'temperature' :  value[0], 'temp_max' : value[1], 'temp_min' : value[2], 'rainfall' : value[3]*30} , ignore_index=True)

    grain_mmx = scaler.fit_transform(grain_data)

    # display(grain_data.tail())
    # display(grain_mmx[:-1])
    # display(grain_mmx[-1])

    X_train, X_test, y_train, y_test = train_test_split(
        grain_mmx[:-len(data)], target_lb, stratify=target_lb, test_size=0.2,
        random_state=2022
    )

    lgb = LGBMClassifier(learning_rate = 0.01, n_estimators = 1000, num_iteration = 5) 

    evals = [(X_test, y_test)]
    lgb.fit(X_train, y_train, eval_set=evals, eval_metric='logloss', verbose=False)


    # print(grain_mmx[-1])
    return_pred = pd.DataFrame()
    for i, value in enumerate(data):
        df_pred = pd.DataFrame(lgb.predict_proba(list(grain_mmx[-(len(data)-i)])), columns=labelencoder.classes_).T
        df_pred.columns = [value[4]]
        # df_pred = df_pred.sort_values(by='확률',ascending=False)
        return_pred = pd.concat([return_pred, df_pred], axis=1)
    return return_pred

def OLS_model_predict(region, month, pred_year):
    df_tmp_region = df_weather[df_weather['조사지역'] == region]
    df_weather_region = df_tmp_region.groupby(['조사지역', '관측일자']).mean()
    df_weather_region = df_weather_region.reset_index(drop=False)

    df_group_mean = df_weather_region.groupby(['연', '월']).mean()
    df_group_mean = df_group_mean.reset_index(drop=False)
    df_month_mean = df_group_mean[df_group_mean['월'] == month]

    for i in df_month_mean.index:
        df_tmp = df_weather_region.loc[(df_weather_region['연'] == df_month_mean.loc[i, '연']) & (df_weather_region['월'] == df_month_mean.loc[i, '월'])]
        temp_max = df_tmp['temp_max'].max()
        temp_min = df_tmp['temp_min'].min()
        df_month_mean.loc[i, 'temp_max'] = temp_max
        df_month_mean.loc[i, 'temp_min'] = temp_min

    pred_list = []
    for value in df_month_mean.columns[2: -2]:
        df_target = df_month_mean[value]
        X_train2 = pd.DataFrame(df_month_mean[['연']], columns = ['연'])
        y_train2 = df_target.values

        # import statsmodels.api as sm
        X_train = sm.add_constant(X_train2)
        model = sm.OLS(y_train2, X_train2).fit()
        pred_value = model.predict(pred_year)
        pred_list.append([region, float(pred_value)])

    # print(f'\t\t\t{region}의 {month}월 {pred_year}의 예측값:')
    # print(pred_list)
    return pred_list






In [ ]:
%time df = predict_6m_all_region_lgb(3, 2100)
for col in df.columns:
    df = df.sort_values(by=col, ascending=False)
    print(col, '\n', df[col].head(3))

CPU times: user 1min 3s, sys: 441 ms, total: 1min 3s
Wall time: 1min 4s
서울 
 wheat      0.066671
coconut    0.058220
banana     0.058210
Name: 서울, dtype: float32
관악산 
 wheat          0.065890
rice           0.058279
kidneybeans    0.057813
Name: 관악산, dtype: float32
춘천 
 wheat     0.066810
lentil    0.058663
maize     0.057711
Name: 춘천, dtype: float32
철원 
 wheat     0.06640
grapes    0.05791
lentil    0.05791
Name: 철원, dtype: float32
속초 
 wheat    0.067379
cocoa    0.059515
maize    0.057883
Name: 속초, dtype: float32
원주 
 wheat     0.066593
lentil    0.058446
maize     0.057494
Name: 원주, dtype: float32
대관령 
 wheat     0.075296
grapes    0.058402
maize     0.057829
Name: 대관령, dtype: float32
태백 
 wheat          0.073527
cocoa          0.062328
kidneybeans    0.057393
Name: 태백, dtype: float32
인제 
 wheat     0.074589
lentil    0.058243
maize     0.057305
Name: 인제, dtype: float32
홍천 
 wheat     0.074350
lentil    0.058004
maize     0.057460
Name: 홍천, dtype: float32
대전 
 wheat      0.067166
co

In [ ]:
%time df_xg = predict_6m_all_region(3, 2022)

for col in df_xg.columns:
    df_xg = df_xg.sort_values(by=col, ascending=False)
    print(col, '\n', df_xg[col].head(3))

CPU times: user 1min 9s, sys: 467 ms, total: 1min 9s
Wall time: 1min 15s
서울 
 wheat          0.065607
kidneybeans    0.057892
grapes         0.057516
Name: 서울, dtype: float32
관악산 
 wheat          0.065893
rice           0.057904
kidneybeans    0.057816
Name: 관악산, dtype: float32
춘천 
 wheat     0.066429
maize     0.064586
lentil    0.058283
Name: 춘천, dtype: float32
철원 
 wheat          0.073012
cocoa          0.057890
kidneybeans    0.057241
Name: 철원, dtype: float32
속초 
 wheat     0.067120
grapes    0.058626
maize     0.058033
Name: 속초, dtype: float32
원주 
 wheat     0.067121
lentil    0.058627
maize     0.058034
Name: 원주, dtype: float32
대관령 
 wheat    0.072875
maize    0.070929
rice     0.064342
Name: 대관령, dtype: float32
태백 
 wheat    0.074154
maize    0.063932
cocoa    0.058281
Name: 태백, dtype: float32
인제 
 wheat     0.074049
grapes    0.057503
lentil    0.057503
Name: 인제, dtype: float32
홍천 
 wheat     0.074283
maize     0.064440
lentil    0.058080
Name: 홍천, dtype: float32
대전 
 wheat    

In [ ]:
%time df_xg = predict_6m_all_region(3, 2100)

for col in df_xg.columns:
    df_xg = df_xg.sort_values(by=col, ascending=False)
    print(col, '\n', df_xg[col].head(3))

CPU times: user 1min 6s, sys: 556 ms, total: 1min 6s
Wall time: 1min 13s
서울 
 wheat      0.066671
coconut    0.058220
banana     0.058210
Name: 서울, dtype: float32
관악산 
 wheat          0.065890
rice           0.058279
kidneybeans    0.057813
Name: 관악산, dtype: float32
춘천 
 wheat     0.066810
lentil    0.058663
maize     0.057711
Name: 춘천, dtype: float32
철원 
 wheat     0.06640
grapes    0.05791
lentil    0.05791
Name: 철원, dtype: float32
속초 
 wheat    0.067379
cocoa    0.059515
maize    0.057883
Name: 속초, dtype: float32
원주 
 wheat     0.066593
lentil    0.058446
maize     0.057494
Name: 원주, dtype: float32
대관령 
 wheat     0.075296
grapes    0.058402
maize     0.057829
Name: 대관령, dtype: float32
태백 
 wheat          0.073527
cocoa          0.062328
kidneybeans    0.057393
Name: 태백, dtype: float32
인제 
 wheat     0.074589
lentil    0.058243
maize     0.057305
Name: 인제, dtype: float32
홍천 
 wheat     0.074350
lentil    0.058004
maize     0.057460
Name: 홍천, dtype: float32
대전 
 wheat      0.067166
c